# This is the Data Science capstone project for Segmenting and Clustering Neighbourhood data in Toronto

In [149]:
import pandas as pd
import numpy as np

#### 1.  Read the wiki html directly into  using pandas read_html. this reads 2 tables present in the html into and gives a list of 2 different dataframes
#### 2.  We need only the first table (dataframe). read the 1st occurance of the list and it gives us the required dataframe

In [150]:
df = pd.read_html(r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header =None)
df = df[0]
df.head()


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### 1.   Get names of indexes for which column Borough has a value of Not assigned
#### 2.   Delete these row indexes from dataFrame

In [151]:
indexNames = df[df['Borough'] == 'Not assigned'].index 
df.drop(indexNames, inplace=True)
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


#### 1. Group by Postcode and Porough and merge the Neighbourhood rows seperated by comma
#### 2. If Neighbourhood is 'Not assigned' then use the value from Borough

In [152]:
df1=df.groupby(['Postcode','Borough'], as_index=False,sort=False).agg( ','.join)
df1.Neighbourhood[df1['Neighbourhood'] == 'Not assigned'] = df1.Borough
print(df1.head(10))


                       

  Postcode           Borough                    Neighbourhood
0      M3A        North York                        Parkwoods
1      M4A        North York                 Victoria Village
2      M5A  Downtown Toronto         Harbourfront,Regent Park
3      M6A        North York  Lawrence Heights,Lawrence Manor
4      M7A      Queen's Park                     Queen's Park
5      M9A         Etobicoke                 Islington Avenue
6      M1B       Scarborough                    Rouge,Malvern
7      M3B        North York                  Don Mills North
8      M4B         East York   Woodbine Gardens,Parkview Hill
9      M5B  Downtown Toronto          Ryerson,Garden District


In [153]:
df1.shape

(103, 3)

## **** This is Part 2 of the Segmenting and clustering neighbourhood data project ****

###### Read the csv file contaning Geospatial data

In [154]:
gscsv = pd.read_csv(r'http://cocl.us/Geospatial_data')
gscsv.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### 1. sort the neighbourhood dataframe by Post code and rename the Postcode column to PostalCode
##### 2. rename the Postal Code column from the csv dataframe to PostalCode

In [155]:
df1.sort_values(by='Postcode',inplace=True)
df1.rename(columns={'Postcode':'PostalCode'},inplace=True)
df1.head(5)
gscsv.rename(columns={'Postal Code':'PostalCode'},inplace=True)
gscsv.head(5)


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### Join the 2 dataframes 

In [158]:
new_df = df1.join(gscsv.set_index('PostalCode'), on='PostalCode')
new_df.reset_index(drop=True,inplace=True)
new_df.head(10)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


## **** This is Part 3 of the Segmenting and clustering neighbourhood data project ****

In [159]:
!conda install -c conda-forge geopy --yes  
from geopy.geocoders import Nominatim  
import json 
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0
import folium
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

###### Check the number of unique places in Borough and slice the dataframe to obtain values for Downtown Toronto

In [177]:
new_df['Borough'].value_counts()
tdf = new_df[new_df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
tdf

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
9,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752


##### Get the latitude and longitude of Downtown Toronto using geolocator and create the map using Folium

In [170]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="tt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.655115, -79.380219.


In [172]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(tdf['Latitude'], tdf['Longitude'], tdf['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

##### 1) Connect to the Four Square API 2) Get the first entry of the tdf in Downtown Toronto (Rosedale) 3) Using the latitude and longitude of rosedal from the tdf, get the "Venue" details using Four Square API 

In [173]:
CLIENT_ID = 'EFOCSLTOZMCE520GGEGFSSBIFXWVTOCOYK1UECFKBQJW55I3' 
CLIENT_SECRET = 'HPP3MJUDROTKJFK22OUMSKU1K50M4SLHPPQTKELTEVVOJTL1' 
VERSION = '20190831'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EFOCSLTOZMCE520GGEGFSSBIFXWVTOCOYK1UECFKBQJW55I3
CLIENT_SECRET:HPP3MJUDROTKJFK22OUMSKU1K50M4SLHPPQTKELTEVVOJTL1


In [181]:
tdf.loc[0,'Neighbourhood']
neighbourhood_latitude = tdf.loc[0, 'Latitude'] 
neighbourhood_longitude = tdf.loc[0, 'Longitude']
neighbourhood_name = tdf.loc[0, 'Neighbourhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


In [182]:
radius = 500
LIMIT = 10

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=EFOCSLTOZMCE520GGEGFSSBIFXWVTOCOYK1UECFKBQJW55I3&client_secret=HPP3MJUDROTKJFK22OUMSKU1K50M4SLHPPQTKELTEVVOJTL1&v=20190831&ll=43.6795626,-79.37752940000001&radius=500&limit=10'

In [183]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d7dc1a42619ee002c1a83ff'},
 'response': {'headerLocation': 'Rosedale',
  'headerFullLocation': 'Rosedale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.6840626045, 'lng': -79.37131878274371},
   'sw': {'lat': 43.675062595499995, 'lng': -79.38374001725632}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bae2150f964a520df873be3',
       'name': 'Mooredale House',
       'location': {'address': '146 Crescent Rd.',
        'crossStreet': 'btwn. Lamport Ave. and Mt. Pleasant Rd.',
        'lat': 43.678630645646535,
        'lng': -79.38009142511322,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.678630645646535,
          'lng': -79.380091425113

In [184]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [185]:
venues = results['response']['groups'][0]['items']
  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Mooredale House,Building,43.678631,-79.380091
1,Rosedale Park,Playground,43.682328,-79.378934
2,Whitney Park,Park,43.682036,-79.373788
3,Alex Murray Parkette,Park,43.678300,-79.382773
4,Milkman's Lane,Trail,43.676352,-79.373842


In [186]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


##### Follow the same approach for all the entries in tdf for Downtown Toronto. Get the venue details for each of the place. To do this, create a function to loop into the entire tdf (Neighbourhood) and get the respective details from Foursquare API 

In [187]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##### Call the defined function to get the "near by"  venue details of all Neighbourhoods in the tdf.

In [188]:
toronto_venues = getNearbyVenues(names=tdf['Neighbourhood'],
                                   latitudes=tdf['Latitude'],
                                   longitudes=tdf['Longitude']
                                  )

Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie


In [189]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",10,10,10,10,10,10
Berczy Park,10,10,10,10,10,10
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",10,10,10,10,10,10
"Cabbagetown,St. James Town",10,10,10,10,10,10
Central Bay Street,10,10,10,10,10,10
"Chinatown,Grange Park,Kensington Market",10,10,10,10,10,10
Christie,10,10,10,10,10,10
Church and Wellesley,10,10,10,10,10,10
"Commerce Court,Victoria Hotel",10,10,10,10,10,10


In [190]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 77 uniques categories.


###### Build a dataframe to capture details of all "near by" venue categories and initialize the same 

In [192]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

print(toronto_onehot)
# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

     Airport  Airport Food Court  Airport Lounge  Airport Terminal  \
0          0                   0               0                 0   
1          0                   0               0                 0   
2          0                   0               0                 0   
3          0                   0               0                 0   
4          0                   0               0                 0   
5          0                   0               0                 0   
6          0                   0               0                 0   
7          0                   0               0                 0   
8          0                   0               0                 0   
9          0                   0               0                 0   
10         0                   0               0                 0   
11         0                   0               0                 0   
12         0                   0               0                 0   
13         0        

,Neighbourhood,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Baby Store,Bakery,...,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


###### Use the group by function to retrieve the mean of all available 'near by' venue counts for each neighbourhood. ex:- Berczy Park does not have airport but have 0.1 steakhouse compared to other locations average

In [193]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Baby Store,Bakery,...,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,...,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0
1,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.1,0.1,0.2,0.1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Cabbagetown,St. James Town",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,"Chinatown,Grange Park,Kensington Market",0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1
6,Christie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Church and Wellesley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0
8,"Commerce Court,Victoria Hotel",0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,"Design Exchange,Toronto Dominion Centre",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### Build a new grouped dataframe based on the frequency of occurance of each category of 'near by' venue and list down the top 5 among those

In [194]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
              venue  freq
0  Asian Restaurant   0.2
1             Plaza   0.1
2             Hotel   0.1
3         Speakeasy   0.1
4       Pizza Place   0.1


----Berczy Park----
               venue  freq
0       Cocktail Bar   0.1
1  French Restaurant   0.1
2       Concert Hall   0.1
3               Park   0.1
4             Museum   0.1


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                venue  freq
0      Airport Lounge   0.2
1             Airport   0.1
2                 Bar   0.1
3     Harbor / Marina   0.1
4  Airport Food Court   0.1


----Cabbagetown,St. James Town----
                 venue  freq
0                 Café   0.2
1   Italian Restaurant   0.1
2  Japanese Restaurant   0.1
3    Indian Restaurant   0.1
4        Jewelry Store   0.1


----Central Bay Street----
                venue  freq
0         Coffee Shop   0.3
1  Italian Restaurant   0.1
2    Ramen Restaurant   0.

##### Get the most common venues (in the descending order of their occurance) for each neighbourhood and create a dataframe to append the 'Neighbourhood' + Most common venues

In [195]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [218]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Asian Restaurant,Pizza Place,Steakhouse,Vegetarian / Vegan Restaurant,Hotel,Concert Hall,Plaza,Speakeasy,Greek Restaurant,Baby Store
1,Berczy Park,Cocktail Bar,Liquor Store,French Restaurant,Vegetarian / Vegan Restaurant,Museum,Farmers Market,Steakhouse,Park,Concert Hall,Beer Bar
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport,Airport Food Court,Airport Terminal,Harbor / Marina,Plane,Bar,Coffee Shop,Boutique,Deli / Bodega
3,"Cabbagetown,St. James Town",Café,Italian Restaurant,Butcher,Restaurant,Diner,Bakery,Japanese Restaurant,Jewelry Store,Indian Restaurant,Farmers Market
4,Central Bay Street,Coffee Shop,Gastropub,Modern European Restaurant,Ramen Restaurant,Sushi Restaurant,Italian Restaurant,Middle Eastern Restaurant,Park,Vietnamese Restaurant,Comic Shop


##### Run K mean clustering model on the resultant dataframe. Set the cluster size as 3. Merge the cluster label (1st cluster, 2nd cluster, 3rd cluster) to the dataframe

In [219]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([2, 2, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [220]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = tdf

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()  

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0,Park,Trail,Playground,Building,Vietnamese Restaurant,Creperie,Cocktail Bar,Coffee Shop,College Gym,Comic Shop
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675,1,Café,Italian Restaurant,Butcher,Restaurant,Diner,Bakery,Japanese Restaurant,Jewelry Store,Indian Restaurant,Farmers Market
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Bubble Tea Shop,Theme Restaurant,Gastropub,Tea Room,Breakfast Spot,Mexican Restaurant,Dance Studio,Ramen Restaurant,Restaurant,Salon / Barbershop
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,1,Gym / Fitness Center,Breakfast Spot,Bakery,Coffee Shop,Pub,Restaurant,Spa,Park,Deli / Bodega,Dessert Shop
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,2,Burrito Place,Comic Shop,Café,Pizza Place,Theater,Thai Restaurant,Tea Room,Clothing Store,Plaza,Ramen Restaurant


In [221]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Segregate the most common venues based on the cluster label (starting from 1 to 3) and display the results for each cluster label. We can observe that the 2nd cluster is having more common venues to explore

In [222]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Park,Trail,Playground,Building,Vietnamese Restaurant,Creperie,Cocktail Bar,Coffee Shop,College Gym,Comic Shop


In [223]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,1,Café,Italian Restaurant,Butcher,Restaurant,Diner,Bakery,Japanese Restaurant,Jewelry Store,Indian Restaurant,Farmers Market
2,Downtown Toronto,1,Bubble Tea Shop,Theme Restaurant,Gastropub,Tea Room,Breakfast Spot,Mexican Restaurant,Dance Studio,Ramen Restaurant,Restaurant,Salon / Barbershop
3,Downtown Toronto,1,Gym / Fitness Center,Breakfast Spot,Bakery,Coffee Shop,Pub,Restaurant,Spa,Park,Deli / Bodega,Dessert Shop
5,Downtown Toronto,1,Coffee Shop,Gastropub,Creperie,Gym,Food Truck,Italian Restaurant,Japanese Restaurant,Restaurant,Middle Eastern Restaurant,College Gym
7,Downtown Toronto,1,Coffee Shop,Gastropub,Modern European Restaurant,Ramen Restaurant,Sushi Restaurant,Italian Restaurant,Middle Eastern Restaurant,Park,Vietnamese Restaurant,Comic Shop
10,Downtown Toronto,1,Coffee Shop,Gym,Bakery,Pub,Restaurant,Beer Bar,Hotel,Gastropub,Café,Food Truck
11,Downtown Toronto,1,Café,Gym / Fitness Center,Restaurant,Gastropub,Museum,Coffee Shop,Pub,Gym,American Restaurant,Baby Store
12,Downtown Toronto,1,Beer Bar,Japanese Restaurant,Restaurant,Dessert Shop,Bookstore,Bar,Bakery,College Gym,Italian Restaurant,French Restaurant
13,Downtown Toronto,1,Café,Bakery,Mexican Restaurant,Organic Grocery,Coffee Shop,Bar,Vietnamese Restaurant,Arts & Crafts Store,Asian Restaurant,College Gym
14,Downtown Toronto,1,Airport Lounge,Airport,Airport Food Court,Airport Terminal,Harbor / Marina,Plane,Bar,Coffee Shop,Boutique,Deli / Bodega


In [224]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,2,Burrito Place,Comic Shop,Café,Pizza Place,Theater,Thai Restaurant,Tea Room,Clothing Store,Plaza,Ramen Restaurant
6,Downtown Toronto,2,Cocktail Bar,Liquor Store,French Restaurant,Vegetarian / Vegan Restaurant,Museum,Farmers Market,Steakhouse,Park,Concert Hall,Beer Bar
8,Downtown Toronto,2,Asian Restaurant,Pizza Place,Steakhouse,Vegetarian / Vegan Restaurant,Hotel,Concert Hall,Plaza,Speakeasy,Greek Restaurant,Baby Store
9,Downtown Toronto,2,Park,Neighborhood,Bubble Tea Shop,Dessert Shop,Deli / Bodega,Supermarket,Sporting Goods Shop,Café,Lake,Salad Place
15,Downtown Toronto,2,Park,Concert Hall,Fountain,Vegetarian / Vegan Restaurant,Thai Restaurant,Tea Room,Museum,Cocktail Bar,Steakhouse,Jazz Club
